In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv("./../config/credentials_my.env")
model_name =  "gpt-4o-mini" # gpt-4o-mini or gpt-4o

print(f'Project Connection String: <...{os.environ["PROJECT_CONNECTION_STRING"][-30:]}>')

Project Connection String: <...1-rg;mmai01-hub-project01-vhdt>


In [2]:
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FilePurpose
from azure.identity import DefaultAzureCredential

In [3]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

In [4]:
project_client.agents.list_agents()

{'object': 'list', 'data': [{'id': 'asst_7XCoUPFAsBV5F7Xqx8YAluB3', 'object': 'assistant', 'created_at': 1736619154, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'code_interpreter': {'file_ids': []}}, 'metadata': {}, 'response_format': 'auto'}, {'id': 'asst_IcNuUuhnf2YQ9fx3d3DBPy9j', 'object': 'assistant', 'created_at': 1736619026, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'code_interpreter': {'file_ids': []}}, 'metadata': {}, 'response_format': 'auto'}, {'id': 'asst_mvHayfUUXBsHv2Q1JEiRt2Km', 'object': 'assistant', 'created_at': 1736618843, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_in

In [5]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai01-rg',
 'project_name': 'mmai01-hub-project01-vhdt'}

In [6]:
from azure.ai.projects.models import CodeInterpreterTool

code_interpreter = CodeInterpreterTool()

print(f"Code interpreter definitions: {code_interpreter.definitions}")
print(f"Code interpreter resources: {code_interpreter.resources}")
print(f"Code interpreter file_ids: {code_interpreter.file_ids}")

Code interpreter definitions: [{'type': 'code_interpreter'}]
Code interpreter resources: {}
Code interpreter file_ids: set()


In [7]:
agent = project_client.agents.create_agent(
    model=model_name,
    name="my-agent",
    instructions="You are a helpful agent",
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources
)

agent.items

<bound method _MyMutableMapping.items of {'id': 'asst_VLBYfenPZxlHnhNS8I6xtlvr', 'object': 'assistant', 'created_at': 1736620341, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'code_interpreter': {'file_ids': []}}, 'metadata': {}, 'response_format': 'auto'}>

In [8]:
# Create a second agent
agent_02 = project_client.agents.create_agent(
    model=model_name,
    name="my-agent",
    instructions="You are a helpful agent",
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources,
)

agent_02.items

<bound method _MyMutableMapping.items of {'id': 'asst_WYCK4dDv34RgD78XWzisEzSS', 'object': 'assistant', 'created_at': 1736620342, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'code_interpreter': {'file_ids': []}}, 'metadata': {}, 'response_format': 'auto'}>

In [9]:
# List all agents
print(project_client.agents.list_agents()['data'])

[{'id': 'asst_WYCK4dDv34RgD78XWzisEzSS', 'object': 'assistant', 'created_at': 1736620342, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'code_interpreter': {'file_ids': []}}, 'metadata': {}, 'response_format': 'auto'}, {'id': 'asst_VLBYfenPZxlHnhNS8I6xtlvr', 'object': 'assistant', 'created_at': 1736620341, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'code_interpreter': {'file_ids': []}}, 'metadata': {}, 'response_format': 'auto'}, {'id': 'asst_7XCoUPFAsBV5F7Xqx8YAluB3', 'object': 'assistant', 'created_at': 1736619154, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0,

In [10]:
project_client.agents.delete_agent(agent_02.id)

{'id': 'asst_WYCK4dDv34RgD78XWzisEzSS', 'object': 'assistant.deleted', 'deleted': True}

In [11]:
# Create a thread
thread = project_client.agents.create_thread()

print(f"Created thread: {thread}")

Created thread: {'id': 'thread_0FoX96TckVSMEOI0mkjbltx0', 'object': 'thread', 'created_at': 1736620344, 'metadata': {}, 'tool_resources': {}}


In [12]:
# Create a message
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="""
    Could you please create a bar chart for the operating profit using the following data and provide the file to me? 
    Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, Company D: $1.8 million
    """,
)
print(f"Messages: {project_client.agents.list_messages(thread_id=thread.id)}")

Messages: {'object': 'list', 'data': [{'id': 'msg_HcbdvB9wTAmVQ00KHMrCo8JI', 'object': 'thread.message', 'created_at': 1736620345, 'assistant_id': None, 'thread_id': 'thread_0FoX96TckVSMEOI0mkjbltx0', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': '\n    Could you please create a bar chart for the operating profit using the following data and provide the file to me? \n    Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, Company D: $1.8 million\n    ', 'annotations': []}}], 'attachments': [], 'metadata': {}}], 'first_id': 'msg_HcbdvB9wTAmVQ00KHMrCo8JI', 'last_id': 'msg_HcbdvB9wTAmVQ00KHMrCo8JI', 'has_more': False}


In [ ]:
%%time

# Run the agent
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)

print(f"Run status: {run.status}.\nRun details: {run}")

In [ ]:
project_client.agents.list_agents()

In [ ]:
# Get messages from the thread
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

In [ ]:
# Get the last message from the sender
last_msg = messages.get_last_text_message_by_sender("assistant")
if last_msg:
    print(f"Last Message: {last_msg.text.value}")

In [ ]:
# messages.file_path_annotations[0].text.split('/')[-1]
messages.file_path_annotations

In [ ]:
for file_path_annotation in messages.file_path_annotations:
    print(f"File Paths: {file_path_annotation}")
    file_name = file_path_annotation.text.split('/')[-1]
    project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name)
    print(f"Saved text file to: {os.getcwd()}\\{file_name}")

In [16]:
# Generate an image file for the bar chart
for image_content in messages.image_contents:
    print(f"Image content: {image_content}")
    file_name = f"{image_content.image_file.file_id}_image_file.png"
    project_client.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
    print(f"Saved image file to: {os.getcwd()}\\{file_name}")

NameError: name 'messages' is not defined

In [18]:
# Delete all agents
for pca in project_client.agents.list_agents()['data']:
    print(f"Deleting agent {pca.name} ({pca.id})...")
    project_client.agents.delete_agent(pca.id)

Deleting agent my-agent (asst_RlNgAOZ58HObEDRb7uDBt9BP)...


# HIC SUNT LEONES